# Getting Started
Continuing from the NEW_USERS.md guide.

## Numerical Solutions to PDEs
This is a field that has a vast amount of research done in it and it is insulting to attempt to summarize it here. Unfortunately, I am a rude person, so I will attempt to do exactly that.

### The Finite Difference Method
The basis of solving PDEs numerically is the idea of discretizing the derivatives and then iterating these discretizations through time (so time is also discretized!). Consider a simple first derivative:

$$\frac{du}{dx}$$

Recall that we can calculate the derivative from first principles as such

$$\frac{du}{dx} = \lim_{h\to 0} \frac{u(x+h) - u(x)}{h}$$

The discretization of a derivative is based off of this concept. Suppose we had an interval $x\in [a,b]$. For some integer $N_x$, let us split up this interval into $N_x+1$ partitions and label the end-points as $[x_i, x_{i+1}]$, $i=0,\dots,N_x$. Then, the entire interval has $N_x+2$ points: a left-most point $x_0$, a right-most point $x_{N_x+1}$, and $N_x$ inner points $x_1, \dots, x_{N_x}$. Let us also define

$$U_i \equiv u(x_i) $$

so that an approximation to the derivative of $U$ can be expressed as

$$\frac{dU_i}{dx} = \frac{U_{i+1} - U_i }{\Delta x}$$

where

$$\Delta x \equiv \frac{x_{N_x+1} - x_0}{N_x+1}$$

As such, $U_{i+1}$ can be thought of as $u(x+h)$. If we were to take $N_x\to\infty$, then this would no longer be an approximation. This method is called the **finite difference method**.

It can be shown (not here) that the second derivative can be written as

$$\frac{d^2U_i}{dx^2} = \frac{U_{i+1} - 2U_i + U_{i-1} }{\Delta x^2}$$

### The Forward Euler Method
The Forward Euler Method is a method used on ordinary differential equations (ODEs). It is quite similar to the finite difference method. Consider the problem

$$ y'(t) = f(t,y(t)), \qquad \qquad y(t_0)=y_0 $$

For some interval $t\in [t_0, t_f]$. For some integer $N_t$, let us split this interval into $N_t-1$ partitions so we have $N_t$ points: a left-most point $t_0$ and a right-most point $t_{N_t-1} = t_f$.

We can approximate the derivative like so:

$$ y_n'(t) = \frac{y_{n+1} - y_n}{\Delta t} $$

where $$\Delta t \equiv \frac{t_f - t_0}{N}$$ Then

$$ f(t,y(t)) = \frac{y_{n+1} - y_n}{\Delta t}$$

which looks just like our finite difference method approach.


### The Heat Equation
Although this equation can be solved analytically, consider the 1D Heat Equation:

$$ \frac{\partial u}{\partial t} = \kappa \frac{\partial^2 u}{\partial x^2} $$

Assuming $x\in [0,L]$ and $t\in[0,T]$, we will use the following initial condition and boundary conditions:

$$ u(x,0) = f(x) $$

$$ u(0,t) = \alpha, \quad u(L,t) = \beta $$

Physically, this set of equations can be thought of as a metal wire whose heat is distributed according to $f(x)$, while the left end of the rod is constantly being set to the temperature $\alpha$ and the right end of the rod is constantly being set to the temperature $\beta$.

First discretizing the equation in space gives us

$$ \frac{du}{dt} = \frac{\kappa}{\Delta x^2} (U_{i+1} - 2U_i + U_{i-1} ) $$

where $i=1,\dots,N_x$. The same "set up" (number of spatial and temporal intervals and points) is used. Now, if we discretize in time, we get

$$ \frac{U^{n+1}_i - U^n_i}{\Delta t} = \frac{\kappa}{\Delta x^2} (U^n_{i+1} - 2U^n_i + U^n_{i-1}) $$

where $n=0,\dots,N_t$. Note that $U^n_i = U(x_i, t_n)$. We can rearrange this equation to get

$$ U^{n+1}_i = U^n_i + K (U^n_{i+1} - 2U^n_i + U^n_{i-1} ) $$

where $K = \frac{\kappa \Delta t}{\Delta x^2} $. Further simplification yields

$$ U^{n+1}_i = (1 - 2K)U^n_i + K (U^n_{i+1} + U^n_{i-1} ) $$ 

(An alteration to this, for the sake of optimization, is to define a constant to be $1 - 2K$ to reduce the number of FLOPs).

You may be wondering how the initial/boundary conditions come into play. We must discretize them as well, so to speak:

$$ U^0_i = f(x_i) \quad i=1,\dots,N_x $$

$$ U^n_0 = \alpha \quad \forall n $$

$$ U^n_{m+1} = \beta \quad \forall n $$

So, we know the initial solution to our problem $(t = t_0)$. To find the solution for all points in time, we must step our initial solution through time. For example, calculating the solution at $t = t_1$:

$$ U^1_0 = \alpha $$

$$ U^1_1 = (1 - 2K)U^0_1 + K(U^0_2 + U^0_0)$$
$$ \vdots $$
$$ U^1_{N_x} = (1 - 2K)U^0_{N_x} + K(U^0_{N_x+1} + U^0_{N_x-1})$$

$$ U^1_{N_x+1} = \beta $$

All of these points can be calculated simply enough. This process is repeated until reaching the endpoint in time.

### Implementation of Solving the 1D Heat Equation Numerically
Solving the heat equation numerically is simple enough, but doing it by hand would be quite awful and many mistakes would be made (the first mistake being the decision to do it by hand in the first place). Let's write a program to do it for us. Before we begin, let's install Python's numerical and plotting libraries. In a terminal, run

```
sudo apt-get install python-numpy python-scipy python-matplotlib
```

To make sure these libraries installed correctly, run an interactive Python shell by running `python` in the terminal, and then attempting to import `numpy`, `scipy` and `matplotlib` respectively. If you're not familiar with how Numpy works, see the section on Numpy.

We'll first implement a pure Python solver, which will run slowly, but it will work! We will be solving the 1D heat equation with the following conditions:

$$ x\in[0,1] \quad t\in[0,300] $$

$$ u(x,0) = \sin(\pi x) $$

$$ u(0,t) = u(1,t) = 0 $$

#### Python Implementation

In [2]:
from __future__ import division # this prevents integer division from occuring
import math, copy, time

# spatial conditions
Nx = 1024           # number of inner x points
x0 = 0              # start
xf = 1              # end (L)
dx = (xf-x0)/(Nx+1) # spatial step size

x = [(x0 + i*dx) for i in xrange(Nx+2)]   # all x points, including endpoints


# temporal conditions
Nt = 10000      # number of time steps
t0 = 0          # start
tf = 300        # end
dt = (tf-t0)/Nt # time step size

t = [(t0 + i*dt) for i in xrange(Nt)]   # all times, including beginning and end


# coefficients
k = 0.0002    # kappa, the diffusitivity constant.
K = 0.01      # the coefficient to the discretized PDE
C = 1 - 2*K   # the coefficient to the first term...

# Notice that we set K = 0.01, instead of  k*dt / dx**2 . This is because it would 
# take a much larger number of time steps or spatial points to make the algorithm 
# stable, so we just cheat and make it stable here.


# initial condition function
def f(x):
    return math.sin(math.pi * x)


# the initial solution (apply f to each spatial point)
u  = [f(x_i) for x_i in x]
un = copy.deepcopy(u)   # solution at next time step


# our updater function
# updates u to the next time step
def update_u(un, u, C, K, Nx):
    for i in xrange(1,Nx+1):
        un[i]= C*u[i] + K*(u[i+1] + u[i-1])
    return un

# begin timer
t_start = time.time()

# loop through time
for n in xrange(1,Nt):
    un = update_u(un, u, C, K, Nx)
    u = un
    
# end timer
t_final = time.time()

print t_final - t_start

2.74682307243


#### Numpy Implementation

In [1]:
from __future__ import division
import time
import numpy as np


# spatial conditions
Nx = 1024                    # number of inner x points
x0 = 0                       # start
xf = 1                       # end
dx = (xf-x0)/(Nx+1)          # spatial step size
x  = np.linspace(x0,xf,Nx+2) # x-axis points


# temporal conditions
Nt = 10000        # number of time steps
t0 = 0            # start
tf = 300          # end
dt = (tf - t0)/Nt # time step size
t  = np.linspace(t0, tf, Nt)


# coefficients
k = 0.0002    # kappa, the diffusitivity constant.
K = 0.01      # the coefficient to the discretized PDE
C = 1 - 2*K   # the coefficient to the first term...


# initial condition function
def f(x):
    return np.sin(np.pi*x)


# initial solution
u  = np.array(f(x), dtype=np.float64)
un = np.empty(Nx+2, dtype=np.float64)


# our updater function
# updates u to the next time step
def update_u(un, u, C, K, Nx):
    un[1:Nx+1] = C*u[1:Nx+1] + K*(u[0:Nx] + u[2:Nx+2])
    return un

# start timer
t_start = time.time()

# Loop over time
for j in range(1,Nt):
    un = update_u(un, u, C, K, Nx)  
    u = un

# end timer
t_final = time.time()

print t_final - t_start

0.107331037521


Notice the incredible speed up using Numpy! This is because a good chunk of Numpy is actually written in C.

#### Numpy + Fortran implementation
Fortran is a very old language that was built for computational mathematics. It's quite speedy. A very kind human being decided to write a Python module that allows for easy communication between Fortran programs and Python programs. What we can do is set up the problem in Python (Numpy), and do the "number-crunching" in Fortran. It takes a bit more code, but it's definitely doable. To do this, we must first install `f2py`, the aforementioned Python module. We can do this using Pip:

```
sudo pip install f2py
```

Now let's write some Fortran code!

In [ ]:
%%file heatFortran.f
      subroutine heatf(un, u, C, K, Nx)
      implicit none
        
      double precision un(Nx), u(Nx)
      double precision C, K
      integer Nx, i

cf2py intent(in) :: un, u, C, K
cf2py intent(hide) :: Nx
cf2py intent(out) :: un

      do i=2,Nx
        un(i) = C*u(i) + K*(u(i+1) + u(i-1))
      enddo
            
      end

Now, we have to compile the code:


In [ ]:
!f2py -c -m heatFortran heatFortran.f

Our Python script will look identical to our Numpy script, but with a different method for updating the solution. So, that being said, let's keep everything the same except let's reset our `u` and `un` variables.

In [10]:
# import our fortran function
from heatFortran import heatf

# initial solution
u  = np.array(f(x), dtype=np.float64)
un = np.zeros(Nx+2, dtype=np.float64)

# start timer
t_start = time.time()

# Loop over time
for j in range(1,Nt):
    un = heatf(un, u, C, K)
    u = un

# end timer
t_final = time.time()

print t_final - t_start

0.0167858600616


Even faster! Awesome. Unfortunately, writing Fortran takes much more time than writing Python/Numpy, so the increase in peformance is balanced out by an increase in the time taken to write the code. This is a big deal in the scientific community.

This alone is pretty good progress, but on a large scale, this does not perform very well. We can improve our runtime by making use of multiple cores on our computers, which will be covered in the next part.

## Appendix
### Using Numpy
Numpy is a very powerful numerical library for Python. Its power comes from its "array" type, which allows for easy indexing and "vectorized operations" (applying operations to an entire set of elements).

For example, consider a list of numbers from 1 to 10. Suppose we want to square every element in this list. Using just Python, this is simple enough:

In [4]:
L = range(1,11)
print L

for i in xrange(len(L)):
    L[i] = L[i]**2
print L

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


This is easy enough to understand. A Numpy implementation would be like this:

In [27]:
import numpy as np

A = np.arange(1,11)      # Numpy's equivalent of range; A is an array.
print A
A = A**2
print A

[ 1  2  3  4  5  6  7  8  9 10]
[  1   4   9  16  25  36  49  64  81 100]


We can apply the squaring operator to every element in our array by simply squaring the array itself! This works for other operations, such as addition and subtraction. Something to notice is that the way these variables are printed is slightly different; lists have commas and Numpy arrays do not.

In [28]:
A = np.arange(1,11)

# add 100 to each element of A
print  A + 100

# subtract 5 from each element of A
print  A - 5

# multiply each element of A by 2
print  2*A

# divide each element of A by 10
print  A/10

[101 102 103 104 105 106 107 108 109 110]
[-4 -3 -2 -1  0  1  2  3  4  5]
[ 2  4  6  8 10 12 14 16 18 20]
[ 0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


Multiplication is a bit different; as a math person, it would make the most sense for the product of two arrays to be equivalent to the dot product, but this is not the case.

In [7]:
# construct arrays from a Python list
A = np.array( [1,2,3,4,5] )
B = np.array( [10,20,30,40,50] )

print A*B

[ 10  40  90 160 250]


Notice that we get an array back, instead of a scalar value. This is an important difference: **arrays are not vectors, they just act like them for some operations**. Arrays always perform operations *element-by-element*. So, the first element of A is multiplied by the first element of B, etc.

Arrays, like lists, can be 2D, and can thus be used to act as matrices. Be careful, though; because of the element-by-element nature of arrays, matrix multiplication will not return the desired result.

In [10]:
v1 = np.array([1,2,3])
v2 = np.array([4,5,6])

print np.dot(v1, v2)    # actual dot product

A = np.array([ [1,0,1], [1,2,3], [1,1,0] ])
B = np.array([ [0,2,2], [4,1,1], [1,5,0] ])

print 'element-by-element multiplication:'
print A*B
print 'actual matrix-matrix multiplication:'
print np.dot(A,B)

32
element-by-element multiplication:
[[0 0 2]
 [4 2 3]
 [1 5 0]]
actual matrix-matrix multiplication:
[[ 1  7  2]
 [11 19  4]
 [ 4  3  3]]


Very different results! This throws many people for a loop (no pun intended), including those who have used Numpy quite a bit.

Another powerful feature of Numpy's arrays is its indexing. For example, to get the third element of the second row of a matrix, we would write:

In [18]:
A = np.array([ [1,2,3], [4,5,6], [7,8,9] ])

print A
print A[1, 2]

[[1 2 3]
 [4 5 6]
 [7 8 9]]
6


That is to say, 2D arrays are indexed like `[row, column]`. This is useful, but what's even more useful is selecting entire rows or columns, or both! To get the third row of this array, we would write:

In [19]:
print A[2]

[7 8 9]


(We do not need to specify columns to select a row). How would we go about selecting a column? For example, to get the second column, we cannot use `A[1]` as that would get the second *row*. Here's how:

In [20]:
print A[:, 1]

[2 5 8]


The `:` means "select all (rows/columns)". We could have accessed the third row of our array using `A[2, :]` (try it out!).

Now, what if we only wanted to select every element except the first one of the second column? If we had a Python list and we wanted to skip the first element (index 0), we would write `SomeList[1:]`. Well, it turns out, Numpy arrays support that as well!

In [22]:
print A[1:, 1]

[5 8]


This allows us to create block matrices easily enough. Let's get the 2x2 bottom right corner of our matrix A:

In [23]:
print A[1:3, 1:3]

[[5 6]
 [8 9]]


Awesome! Easy enough indeed.

Numpy also has many, *many* built-in features, such as linear system solvers ($A\vec{x} = \vec{b}$), random number generators, ODE solvers, etc. I'll list a few useful features:

In [43]:
# a 5-by-4 matrix of 1s, which are created as 64-bit floats
np.ones( (5, 4), dtype=np.float64 )

# a 2-by-2 matrix of 0s, which are created as 32-bit ints
np.zeros( (2,2), dtype=np.int32 )

# shifting the rows and columns of matrices
A = np.array([ [1,2,3], [4,5,6], [7,8,9] ])
print A
# shift rows down
print np.roll(A, 1, 0)     # arguments: array, shift amount, axis (0 = rows, 1 = cols)
# shift rows up
print np.roll(A, -1, 0)
# shift columns to the right
print np.roll(A, 1, 1)
# shift columns to the left
print np.roll(A, -1, 1)
print 

# allocate memory to a 10-by-10 matrix of 64-bit floats
B = np.empty( (10, 10), dtype=np.float64 )

# create an array of 20 evenly spaced numbers in interval [2,5] (inclusive)
np.linspace(2, 5, 20)

# mean of an array
np.mean(A)

# "flatten" an array (convert a n-D array to 1D)
print A.flatten()
print


# reshape a 4-by-4 array to a 2-by-8 array
C = np.array([ [1,2,3,4], [5,6,7,8], [9,10,11,12], [13,14,15,16] ])
print C
print C.reshape(2, 8)
print 


# vertically merge a 4-by-4 array with a 2-by-4 array
D = np.array([ [1,1,1,1], [0,0,0,0] ])
print np.vstack([C, D])                    # notice that we put C and D inside a list!
print


# horiziontally merge a 4-by-4 array with a 4-by-2 array
E = np.array([ [1,1], [0,0], [2,2], [3,3] ])
print np.hstack([C, E])

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[7 8 9]
 [1 2 3]
 [4 5 6]]
[[4 5 6]
 [7 8 9]
 [1 2 3]]
[[3 1 2]
 [6 4 5]
 [9 7 8]]
[[2 3 1]
 [5 6 4]
 [8 9 7]]

[1 2 3 4 5 6 7 8 9]

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]]

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]
 [ 1  1  1  1]
 [ 0  0  0  0]]

[[ 1  2  3  4  1  1]
 [ 5  6  7  8  0  0]
 [ 9 10 11 12  2  2]
 [13 14 15 16  3  3]]
